This notebook is mainly to test out teamwork api, its nothing permanent

In [ ]:
import teamwork_client
import json
import datetime
from datetime import timedelta
from zoneinfo import ZoneInfo
from dotenv import load_dotenv
import os

In [ ]:
def print_response(response):
    if 200 <= response.status_code < 300:
        try:
            formatted_json = json.dumps(response.json(), indent=4)
            print(formatted_json)
        except ValueError:
            print("Response content is not in JSON format.")
    else:
        print(f"Failed to retrieve data: {response.status_code} - {response.text}")

In [ ]:
def format_datetime(time_str):
    today = datetime.datetime.now().date()
    time = datetime.datetime.strptime(time_str, '%H:%M:%S').time()
    datetime_obj = datetime.datetime.combine(today, time)
    return datetime_obj.strftime('%Y-%m-%dT%H:%M:%S')

In [ ]:
def est_to_utc(date_str, time_str):
    est_tz = ZoneInfo("America/New_York")
    today = datetime.datetime.now().date()
    est_time = datetime.datetime.strptime(f"{date_str} {time_str}", '%Y-%m-%d %I:%M%p').replace(tzinfo=est_tz)
    utc_time = est_time.astimezone(ZoneInfo('UTC'))
    return utc_time.strftime('%Y-%m-%d'), utc_time.strftime('%H:%M:%S')

In [ ]:
#100% GPT
def get_workweek_dates(start_date_str):
    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    days_to_monday = (start_date.weekday() - 0) % 7
    monday = start_date - timedelta(days=days_to_monday)
    return [monday + timedelta(days=i) for i in range(5)]

In [ ]:
load_dotenv()

In [ ]:
apiToken = os.getenv('TEAMWORK_API')
baseUrl = os.getenv('TEAMWORK_BASEURL')

In [ ]:
PROJECT_NAME = 'Resonant Solutions'
TASK_LISTS = {
    "Labourly": ["Coding", "Meetings"],
    "Admin (RS Core)": ["Meetings & Communications"]
}

In [ ]:
client = teamwork_client.TeamworkClient(baseUrl, apiToken)

In [ ]:
query_param ={
    'searchTerm' : PROJECT_NAME
}
response_project = client.get_projects(query_param)

In [ ]:
if(response_project.status_code != 200):
    raise SystemExit("Exiting due to unsuccessful response. Failed to retrieve data: {response_project.status_code} - {response_project.text}")


project_data = response_project.json().get('projects', [])
projectId = None

for project in project_data:
    if(project['name'] == PROJECT_NAME):
        projectId = project['id']

if(projectId):
    print(f"Project ID for '{PROJECT_NAME}': {projectId}")
else:
    print(f"Project with name '{PROJECT_NAME}' not found.")

In [ ]:
response_task_list = client.get_tasks_list(projectId=projectId)

In [ ]:
if(response_task_list.status_code != 200):
    raise SystemExit("Exiting due to unsuccessful response. Failed to retrieve data: {response_task_list.status_code} - {response_task_list.text}")


task_list_data = response_task_list.json().get('tasklists', [])
task_list_map = {}

for task_list in task_list_data:
    if task_list['name'] in TASK_LISTS:
        task_list_map[task_list['name']] = task_list['id']

In [ ]:
task_map = {}

for task_list_name, task_list_id in task_list_map.items():
    response_task = client.get_tasks_from_tasks_list(task_list_id, None)
    
    if response_task.status_code != 200:
        raise SystemExit(f"Exiting due to unsuccessful response for {task_list_name}. Failed to retrieve data: {response_task.status_code} - {response_task.text}")
    
    tasks_data = response_task.json().get('tasks', [])
    for task in tasks_data:
        if task['name'] in TASK_LISTS[task_list_name]:
            task_map[task['name']] = task['id']

if task_map:
    print("Task ID mappings found:")
    for task_name, task_id in task_map.items():
        print(f"'{task_name}': {task_id}")
else:
    print("No matching tasks found.")


In [ ]:
required_tasks = ["Coding", "Meetings", "Meetings & Communications"]
if all(task in task_map for task in required_tasks):
    workweek_start = "2025-03-31"
    workweek_dates = get_workweek_dates(workweek_start)
    
    time_entries = [
        {
            "task_name": "Coding",
            "hours": 7,
            "minutes": 0,
            "start_time_est": "10:00AM",
            "description": "Daily coding work"
        },
        {
            "task_name": "Meetings",
            "hours": 0,
            "minutes": 30,
            "start_time_est": "9:30AM",
            "description": "Daily team meeting"
        },
        {
            "task_name": "Meetings & Communications",
            "hours": 0,
            "minutes": 30,
            "start_time_est": "9:00AM",
            "description": "Daily communication sync"
        }
    ]

    for work_date in workweek_dates:
        date_str = work_date.strftime('%Y-%m-%d')
        print(f"\nProcessing time entries for {date_str}")
        
        for entry in time_entries:
            task_id = task_map[entry["task_name"]]
            utc_date, utc_time = est_to_utc(date_str, entry["start_time_est"])
            
            time_payload = {
                "timelog": {
                    "description": entry["description"],
                    "hours": entry["hours"],
                    "isBillable": False,
                    "isUtc": True,
                    "minutes": entry["minutes"],
                    "projectId": projectId,
                    "taskId": task_id,
                    "date": utc_date,
                    "time": utc_time
                }
            }
            
            response_time_entries = client.post_time_entry_from_task(task_id, time_payload)
            print(f"Time entry for {entry['task_name']} on {utc_date}:")
            #print_response(response_time_entries)
else:
    missing = [task for task in required_tasks if task not in task_map]
    print(f"Cannot create time entries. Missing task IDs: {missing}")